<a href="https://colab.research.google.com/github/MollySQuinn/interactive_unexpected_events/blob/main/interactive_plots_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up
Left to do: 
* fix the spacing for plots with only one brace

In [1]:
!pip install jupyter-dash dash-bootstrap-components

In [2]:
#some basic imports
import os
import pandas as pd
import numpy as np

#plot-specific imports
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

#dashboard imports
from jupyter_dash import JupyterDash
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  app.launch_new_instance()


## Define Functions to Make the Plots and Draw Braces

In [3]:
# draw braces on the dot plots to nicely label the proportion/percent of responses
def draw_plotly_brace(fig, yspan, ylim, xlim, xx, spec_color, direction="left"):
    """Draws an annotated brace on the axes."""
    ymin, ymax = yspan
    yspan = ymax - ymin
    ax_ymin = 0
    ax_ymax = ylim
    yax_span = ax_ymax - ax_ymin

    xmin = 0
    xmax = xlim
    xspan = xmax - xmin
    resolution = int(yspan/yax_span*100)*2+1 # guaranteed uneven
    beta = 300./yax_span # the higher this is, the smaller the radius

    y = np.linspace(ymin, ymax, resolution)
    y_half = y[:int(resolution/2)+1]
    if direction == "left":
        x_half_brace = -(
            1/(1.+np.exp(-beta*(y_half-y_half[0])))
                        + 1/(1.+np.exp(-beta*(y_half-y_half[-1])))
                       )
    else:
        x_half_brace = (
            1/(1.+np.exp(-beta*(y_half-y_half[0])))
                        + 1/(1.+np.exp(-beta*(y_half-y_half[-1])))
                       )
    
    x = np.concatenate((x_half_brace, x_half_brace[-2::-1]))
    x = xx + (.05*x - .01)*xspan # adjust vertical position
    
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        line = dict(color=spec_color)
        
    )
                 )


In [4]:
#define a function that gets the labels
def get_hover_text(data_for_labels, material, cond, name):
    hover = [] 
    if 'val_code' in data_for_labels.columns:
        dataframe = data_for_labels[
        (data_for_labels['material']==material) &
        (data_for_labels['cond']==cond) &
        (data_for_labels['val_code']==name)]
    elif 'var_feature2' in data_for_labels:
        dataframe = data_for_labels[
        (data_for_labels['material']==material) &
        (data_for_labels['cond']==cond) &
        (data_for_labels['best_senti']==name)]
    else:
        dataframe = data_for_labels[
        (data_for_labels['material']==material) &
        (data_for_labels['var_feature1']==cond) &
        (data_for_labels['best_senti']==name)]
    for item in dataframe['answer_text'].str.wrap(30).apply(lambda x: x.replace('\n', '<br>')):
            hover.append(item)
        
    return hover

In [5]:
# Make a valence-specific Dot Plot.
def val_freq_plot(data, material, cond, data_for_labels, Xlim, title='', x_block_size=20, y_block_size=4, gap_size=0.5, w=18, h=12.5, brace_space_left=0,brace_space_right=1):
    ds = data.loc[material,cond]
    ds = ds.fillna(0)
    symbols = {'neither':['circle-open','black'],
               'neg':['circle','red'],
               'pos':['circle','yellowgreen']}  # 😀
    
#     print(sum(ds))

    braces_lim={}
    annot = {}
    
    Xlim = Xlim
    Ylim = (sum(ds)//Xlim)
    Ylim = Ylim + (Ylim//y_block_size)*gap_size

        
    Ypos = 0
    Xpos = 0 ##change to zero for upwards
    series = []

    
    ygaps=np.arange(y_block_size, Ylim, y_block_size + gap_size)#[4,8+1,12+2,16+3,20+4,24+5,28+6,32+7,36+8,40+9]:  #43+10=53

    braces_lim={}

    if x_block_size < Xlim:
        Xlim = Xlim + (Xlim//x_block_size)*gap_size
        xgaps=np.arange(x_block_size, Xlim, x_block_size+gap_size)
        print(xgaps)
    else:
        xgaps=np.array([])
    
    
    for name, count in ds.iteritems():
        x = []
        y = []        
#         first add name1 then name2
        hover = get_hover_text(data_for_labels, material, cond, name)
        
        for j in range(0, int(count)):

            if Xpos in xgaps:
                Xpos += gap_size
                
            if Xpos == Xlim:
                Xpos = 0
                Ypos += 1 ##change to positive for upwards
            
            if Ypos in ygaps:
                Ypos += gap_size
                
            y.append(Ypos)
            x.append(Xpos)
            Xpos += 1
            
            if Xpos == Xlim:
                Xpos = 0
                Ypos += 1 ##change to positive for upwards

        if len(y):
            braces_lim[name]=(min(y)-0.5,max(y)+0.5)
        else:
            continue


        
        annot[name]=int(round((100*(count/ds.sum()))))
        
        series.append(go.Scatter(x=x, y=y,
                                 hovertext = hover,
                                 hoverinfo="text",
    #                              mode='text', 
    #                              text=[symbols[name][0]]*len(x),
    #                              textfont= {
    #                                     'size': 30,
    #                                     'color': symbols[name][1]
    #                                   },
                                 mode='markers',
                                 marker={'symbol': symbols[name][0], 
                                         'size': 20, 
                                         'color':symbols[name][1],
                                        'line':dict(
                                            color='black',
                                            width=0.4
                                        )
                                         },
                                 name=f'{name} ({count})'
                                )
                            )


    fig = go.Figure(dict(data=series, layout=go.Layout(
    #     title={'text': title, 'x': 0.5, 'xanchor': 'center'},
        paper_bgcolor='rgba(255,255,255,1)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(showgrid=False,zeroline= False, showline=False, visible=False, showticklabels=False),
        yaxis=dict(showgrid=False,zeroline= False, showline=False, visible=False, showticklabels=False),
    )))

    # ## Create the curly brace, thanks to https://stackoverflow.com/a/61454455 and https://stackoverflow.com/a/53383764/9600186 ##
    if 'neg' in braces_lim:
        draw_plotly_brace(fig=fig, yspan=(braces_lim['neg']), ylim=Ylim, xlim=Xlim, xx=brace_space_left, direction="left", spec_color='red')
        fig.add_annotation(text=f"{annot['neg']}%",
                                    x=-2, y=sum(braces_lim['neg'])/2,
                                    showarrow=False,
                                    textangle=-90,
                           font=dict(
                                    family="Arial",
                                    size=16,
                                )
                            )
    if 'pos' in braces_lim:
        draw_plotly_brace(fig=fig, yspan=(braces_lim['pos']), ylim=Ylim, xlim=Xlim, xx=Xlim-brace_space_right, direction="right",spec_color='yellowgreen')
        fig.add_annotation(text=f"{annot['pos']}%",
                            x=Xlim+1, y=sum(braces_lim['pos'])/2,
                            showarrow=False,
                            textangle=-90,
                          font=dict(
                                    family="Arial",
                                    size=16,
                                )
                          )

        
    fig.update_xaxes(
        scaleanchor = "y",
        scaleratio = 1,
    )
    
    fig.update_yaxes(
        range=[-1, Ylim+1], 
        constrain='domain'
      )
    
    fig.update_layout(
        title=title,
        width=Xlim*w,
        height=Ylim*h,
        showlegend=False,
    #remove some margin so the graph fits nicer in latex
        margin_l=0, #left margin
        margin_r=0, #right margin
        margin_b=0, #bottom margin
        margin_t=30, #top margin
    )

    
#     print(ygaps)
#     print(Xlim)
#     print(Ylim)
#     print(fig.layout)
#     fig.write_image(file=f"3_output/{title}_fig.svg")
    return fig

# Experiment 1: Predicting Unexpected Events

In [6]:
expt1 = pd.read_csv("https://raw.githubusercontent.com/MollySQuinn/interactive_unexpected_events/main/0_data/expt1_master_data.csv")
expt1.head()

,Unnamed: 0,expt,cond,var_feature1,user_id,material,material_variant,answer_text,rater_order,all_ans_codes,...,all_obj_js,best_senti,best_obj,best_ans,order,comment,pos,neg,neither,resolve
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,1,expt1,cond1a,UNEXP,54468870fdf99b648b6a5c69,alan_plane,non-goal,He has forgotten his passport,mk kc mq,ap_neg_ans3 ap_neg_ans3 ap_neg_ans3,...,neither_obj neither_obj neither_obj,neg,neither_obj,ap_neg_ans3,to_do,NaN,0,1,0,0
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,3,expt1,cond1a,UNEXP,561f168fed6e5a000fc7e90c,alan_plane,non-goal,His flight is delayed.,mk kc mq,ap_neg_ans6 ap_neg_ans6 ap_neg_ans6,...,goal_obj neither_obj goal_obj,neg,goal_obj,ap_neg_ans6,to_do,NaN,0,1,0,0
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [7]:
# get set of materials
expt1_materials = expt1['material'].dropna().unique()
expt1_materials

array(['alan_plane', 'anna_interview', 'bob_job', 'karen_bus',
       'peter_college', 'michael_tea', 'john_party', 'mary_food',
       'katie_kitten', 'sam_driving', 'louise_shopping', 'sean_call',
       'bill_holiday', 'robert_essay', 'sally_wine', 'rebecca_swimming',
       'lucy_loan', 'edith_exam', 'belinda_meeting', 'steve_gardening'],
      dtype=object)

In [8]:
expt1_scenarios = pd.read_csv("https://raw.githubusercontent.com/MollySQuinn/interactive_unexpected_events/main/0_data/materials/material_prompts_expt_1_2.csv")
expt1_scenarios.head()

,material,sent1,sent2,sent3
0,alan_plane,"Alan has a plane to catch, that is departing f...","In the taxi to the airport, he reads a magazin...","At the airport, Alan checks his bags in at the..."
1,anna_interview,Anna has an important job interview to attend ...,"The night before, when she gets home from work...","Going to bed, Anna sets her alarm clock for 7am."
2,belinda_meeting,Belinda is on a plane going to Los Angeles for...,Looking out the plane window she is thinking f...,"When Belinda reaches Los Angeles, she takes a ..."
3,bill_holiday,Bill is looking forward to going on his summer...,He cooks himself a large breakfast on the day ...,"Then, Bill packs his passport in his suitcase ..."
4,bob_job,Bob wants to quit his job because he is not cr...,"On arriving at the office, he turns on his com...","At coffee-time, Bob goes to talk to his boss a..."


In [9]:
expt1_dict_scenarios = {}
for mat in expt1_materials:
    expt1_dict_scenarios[mat]="\n\n".join(expt1_scenarios[expt1_scenarios['material']==mat][['sent1','sent2','sent3']].values.tolist()[0])

In [10]:
# get answer frequencies by material, valence condition, and valence label
expt1_material_valence = expt1.groupby(['material','var_feature1','best_senti']).count()['user_id'].reset_index()
#expt3_material_valence.head()

In [11]:
expt1_material_valence_pivot = expt1_material_valence.pivot_table(index = ['material','var_feature1'], 
                                                            columns='best_senti', 
                                                            values='user_id')
expt1_materials_forChi = expt1_material_valence_pivot[['neg','pos']]
expt1_materials_forChi.head()

best_senti                    neg   pos
material       var_feature1            
alan_plane     GOAL-FAIL     30.0   NaN
               UNEXP         27.0   5.0
               UNEXP-BAD     32.0   NaN
               UNEXP-GOOD     1.0  32.0
anna_interview GOAL-FAIL     29.0   1.0

In [12]:
val_freq_plot(expt1_materials_forChi, 'louise_shopping', 'UNEXP-BAD', expt1,
                  10,
#                   title= str(mat+" "+cond), 
                  x_block_size=10, 
                  y_block_size=10, 
                  gap_size=0.5, 
                  w=35, 
                  h=50,
                  brace_space_left=-0.3,
                  brace_space_right=.5)

# Experiment 2: Bizarre Unexpected

In [13]:
expt2 = pd.read_csv("https://raw.githubusercontent.com/MollySQuinn/interactive_unexpected_events/main/0_data/expt2_master_data.csv")
expt2['cond'] = expt2['var_feature1'] + "-" + expt2['var_feature2']
expt2.head()

,Unnamed: 0,material,cond,expt,var_feature1,var_feature2,var_feature3,var_feature4,answer_text,user_id,...,best_senti,best_obj,pos,neg,neither,resolve,goal_obj,non_goal_obj,neither_obj,both_objs
0,0,alan_plane,UNEXP-BAD-bizarre,expt2,UNEXP-BAD,bizarre,reverse,goal,His luggage ends up in LA,5a0d9b95087f2e0001eaea6c,...,neg,goal_object,0,1,0,0,1,0,0,0
1,1,alan_plane,UNEXP-BAD-bizarre,expt2,UNEXP-BAD,bizarre,control,goal,his bags are over weight,599b56790ed7ae0001991d4d,...,neg,goal_object,0,1,0,0,1,0,0,0
2,2,alan_plane,UNEXP-BAD-normal,expt2,UNEXP-BAD,normal,control,non-goal,"On arrival in New York, his bag is nowhere to ...",5b1552789663ee0001463aff,...,neg,goal_object,0,1,0,0,1,0,0,0
3,3,alan_plane,UNEXP-normal,expt2,UNEXP,normal,reverse,non-goal,The bags went missing and were put on another ...,598f5361ad9ab80001a01bed,...,neg,goal_object,0,1,0,0,1,0,0,0
4,4,alan_plane,UNEXP-BAD-bizarre,expt2,UNEXP-BAD,bizarre,control,goal,His bags end up elsewhere,5b0eacd3e9270900013bee22,...,neg,goal_object,0,1,0,0,1,0,0,0


In [14]:
# get set of materials
expt2_materials = expt2['material'].dropna().unique()
expt2_materials

array(['alan_plane', 'anna_interview', 'belinda_meeting', 'bill_holiday',
       'bob_job', 'edith_exam', 'john_party', 'karen_bus', 'mary_food',
       'katie_kitten', 'louise_shopping', 'lucy_loan', 'michael_tea',
       'peter_college', 'rebecca_swimming', 'robert_essay', 'sally_wine',
       'sean_call', 'sam_driving', 'steve_gardening'], dtype=object)

In [15]:
# get answer frequencies by material, valence condition, and valence label
expt2_material_valence = expt2.groupby(['material','cond','best_senti']).count()['user_id'].reset_index()
#expt3_material_valence.head()

In [16]:
expt2_material_valence_pivot = expt2_material_valence.pivot_table(index = ['material','cond'], 
                                                            columns='best_senti', 
                                                            values='user_id')
expt2_materials_forChi = expt2_material_valence_pivot[['neg','pos']]
expt2_materials_forChi.head()

best_senti                      neg   pos
material   cond                          
alan_plane UNEXP-BAD-bizarre   38.0   1.0
           UNEXP-BAD-normal    38.0   1.0
           UNEXP-GOOD-bizarre   NaN  39.0
           UNEXP-GOOD-normal    NaN  40.0
           UNEXP-bizarre       26.0   9.0

In [17]:
val_freq_plot(expt2_materials_forChi, 'louise_shopping', 'UNEXP-BAD-normal', expt2,
                  10,
#                   title= str(mat+" "+cond), 
                  x_block_size=10, 
                  y_block_size=10, 
                  gap_size=0.5, 
                  w=35, 
                  h=50,
                  brace_space_left=-0.3,
                  brace_space_right=.5)

# Experiment 3: _a priori_ Valenced Materials

## Get data, set up materials list, etc.

In [18]:
expt3 = pd.read_csv("https://raw.githubusercontent.com/MollySQuinn/interactive_unexpected_events/main/0_data/expt3_master_post_resolve.csv")
expt3.head()

,Unnamed: 0,user_id,answer_text,expt,ans_code,ans_count,val_code,val_count,material,cond
0,0,2.216.190.219,He is delayed in traffic.,expt3_2,sd_neg_ans1,1.0,neg,1.0,sam_driving,neg_mat
1,1,2.216.190.219,He is involved in an accident.,expt3_2,pc_neg_ans1,1.0,neg,1.0,peter_college,neg_mat
2,2,2.216.190.219,He is offered a promotion.,expt3_2,bj_pos_ans2,1.0,pos,1.0,bob_job,pos_mat
3,3,2.216.190.219,"His computer crashes and he loses the work,.",expt3_2,re_neg_ans2,1.0,neg,1.0,robert_essay,neg_mat
4,4,2.216.190.219,She eats the ;pizza crusts.,expt3_2,other,1.0,neg,1.0,mary_food,neg_mat


In [19]:
# get set of materials
materials = expt3['material'].dropna().unique()
materials

array(['sam_driving', 'peter_college', 'bob_job', 'robert_essay',
       'mary_food', 'belinda_meeting', 'sally_wine', 'lucy_loan',
       'steve_garden', 'louise_shopping'], dtype=object)

In [20]:
scenarios = pd.read_csv("https://raw.githubusercontent.com/MollySQuinn/interactive_unexpected_events/main/0_data/materials/Expt3_Materials.csv", encoding='latin-1')
scenarios.head()

,Material,Sent_1,Sent_2_POS,Sent_2_NEG,Sent_3,Comment
0,anna_interview,Anna has an important job interview to attend ...,She is feeling confident and thinks she will g...,She is feeling nervous and thinks she will not...,"Going to bed, Anna sets her alarm clock for 7am.",Sent2-change only
1,alan_plane,"Alan has a plane to catch, that is departing f...","He is looking forward to his flight, because h...","He is worried about his flight, because he is ...","At the airport, Alan checks his bags in at the...",Sent2-change only
2,belinda_meeting,Belinda is on a plane going to Los Angeles for...,She loves these meetings in Los Angeles becaus...,She hates these meetings in Los Angeles becaus...,"When Belinda reaches Los Angeles, she takes a ...",Sent2-change only
3,bill_holiday,"Bill is going on his summer holidays, as it is...",He wakes up refreshed after having a good nigh...,He wakes up exhausted after having a bad night...,Bill packs his passport in his suitcase for th...,Sent1-change Sent2-change
4,bob_job,Bob has been working for an IT company for thr...,He is happy in his job and his hard work is w...,He is unhappy in his job and his hard work is ...,"At coffee-time, Bob goes to talk to his boss a...",Sent1-change Sent2-change


In [21]:
dict_scenarios = {}
for material in materials:
    dict_scenarios[material] = {'pos':"\n\n".join(scenarios[scenarios['Material']==material][['Sent_1','Sent_2_POS','Sent_3']].values.tolist()[0]),
                                'neg':"\n\n".join(scenarios[scenarios['Material']==material][['Sent_1','Sent_2_NEG','Sent_3']].values.tolist()[0])}


In [22]:
# get answer frequencies by material, valence condition, and valence label
expt3_material_valence = expt3.groupby(['material','cond','val_code']).count()['user_id'].reset_index()
#expt3_material_valence.head()

In [23]:
expt3_material_valence_pivot = expt3_material_valence.pivot_table(index = ['material','cond'], 
                                                            columns='val_code', 
                                                            values='user_id')
expt3_materials_forChi = expt3_material_valence_pivot[['neg','pos']]
expt3_materials_forChi.head()

val_code                  neg   pos
material        cond               
belinda_meeting neg_mat  24.0  24.0
                pos_mat  42.0   7.0
bob_job         neg_mat  16.0  34.0
                pos_mat  20.0  30.0
louise_shopping neg_mat  26.0  25.0

In [24]:
val_freq_plot(expt3_materials_forChi, 'louise_shopping', 'pos_mat', expt3,
                  10,
#                   title= str(mat+" "+cond), 
                  x_block_size=10, 
                  y_block_size=10, 
                  gap_size=0.5, 
                  w=35, 
                  h=45,
                  brace_space_left=-0.3,
                  brace_space_right=.5)

# DASH

In [25]:
# suppress logs unless error
import logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

In [26]:
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.css.append_css(
    {'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

app.layout = html.Div(children=[

    # these dropdowns need to change the graphs,
    # and also the material scenarios

    #Start Experiment 1
    html.Div([
        html.H1('Experiment 1 - Predicting the Unexpected',
                style={'textAlign': 'center'}),
        html.Div(dcc.Markdown('''Participants read a material scenario, 
        then responded to one of the following four questions. 
        Each dot represents a response. Hover over it to see the text response.'''
                              ),
                 style={'marginBottom': 25}),
        html.Label('Select a material scenario:'),
        dcc.Dropdown(id='expt1_material_dropdown',
                     options=[{
                         'label': i,
                         'value': i
                     } for i in expt1_materials],
                     value='louise_shopping',
                     style={'marginBottom': 25}),
        html.H2('Material'),
        html.Div(id='expt1_mark', style={'marginBottom': 25}),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.H3('Something unexpected occurred.'),
                        dcc.Markdown('What do you think happened?'),
                        #                       html.Div(id='unexp_mark'),
                        html.Div(dcc.Graph(id='unexp_graph'))
                    ],
                    width=6,
                    style={'marginBottom': 25}),
                dbc.Col(
                    [
                        html.H2(
                            'But {the actor} did not {achieve their goal}'),
                        dcc.Markdown('What do you think happened?'),
                        #                       html.Div(id='goal_fail_mark'),
                        html.Div(dcc.Graph(id='goal_fail_graph'))
                    ],
                    width=6),
            ],
            style={'marginBottom': 25}),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.H2('Something good and unexpected occurred.'),
                        dcc.Markdown('What do you think happened?'),
                        #                       html.Div(id='good_mark'),
                        html.Div(dcc.Graph(id='unexp_good_graph'))
                    ],
                    width=6),
                dbc.Col(
                    [
                        html.H2('Something bad and unexpected occurred.'),
                        dcc.Markdown('What do you think happened?'),
                        #                       html.Div(id='bad_mark'),
                        html.Div(dcc.Graph(id='unexp_bad_graph'))
                    ],
                    width=6),
            ], )
    ]),

    #Put Experiment 2 Here
    html.Div([
        html.H1('Experiment 2 - Unexpected and Bizarre',
                style={
                    'textAlign': 'center',
                    'marginTop': 50
                }),
        html.Div(dcc.Markdown('''Participants read a material scenario, 
        then responded to one of the following six questions:'''),
                 style={'marginBottom': 25}),
        html.Label('Select a material scenario:'),
        dcc.Dropdown(id='expt2_material_dropdown',
                     options=[{
                         'label': i,
                         'value': i
                     } for i in expt2_materials],
                     value='louise_shopping',
                     style={'marginBottom': 25}),
        html.Div(id='expt2_mark', style={'marginBottom': 25}),
        #         dbc.Row( [dbc.Col([html.H2('Normal')],
        #                     width=6),
        #                   dbc.Col([html.H2('Bizarre')],
        #                     width=6)
        #                  ]
        #         ),
        dbc.Row([
            dbc.Col([html.H2('Normal instruction')], width=6),
            dbc.Col([html.H2('Bizarre instruction')], width=6),
        ],
                style={'marginBottom': 25}),
        dbc.Row([
            dbc.Col([
                html.H3('Something unexpected occurred.'),
                html.Div(dcc.Graph(id='unexp_norm_graph'))
            ],
                    width=6),
            dbc.Col([
                html.H3('Something unexpected and bizarre occurred.'),
                html.Div(dcc.Graph(id='unexp_bizarre_graph'))
            ],
                    width=6),
        ],
                style={'marginBottom': 25}),
        dbc.Row([
            dbc.Col([
                html.H3('Something good and unexpected occurred.'),
                html.Div(dcc.Graph(id='unexp_good_norm_graph'))
            ],
                    width=6),
            dbc.Col([
                html.H3('Something good, unexpected and bizarre occurred'),
                html.Div(dcc.Graph(id='unexp_good_bizarre_graph'))
            ],
                    width=6),
        ],
                style={'marginBottom': 25}),
        dbc.Row([
            dbc.Col([
                html.H3('Something bad and unexpected occurred.'),
                html.Div(dcc.Graph(id='unexp_bad_norm_graph'))
            ],
                    width=6),
            dbc.Col([
                html.H3('Something bad, unexpected and bizarre occurred'),
                html.Div(dcc.Graph(id='unexp_bad_bizarre_graph'))
            ],
                    width=6),
        ],
                style={'marginBottom': 25})
    ]),

    #Start Experiment 3
    html.Div([
        html.H1('Experiment 3 - Valenced Materials',
                style={
                    'textAlign': 'center',
                    'marginTop': 50
                }),
        html.Div(dcc.Markdown(
            '''Participants read either a positive or negative version of the material scenario, 
        then responded to the question, "Something unexpected occurred. What do you think happened?"'''
        ),
                 style={'marginBottom': 25}),
        html.Label('Select a material scenario:'),
        dcc.Dropdown(id='expt3_material_dropdown',
                     options=[{
                         'label': i,
                         'value': i
                     } for i in materials],
                     value='louise_shopping',
                     style={'marginBottom': 25}),
        dbc.Row([
            dbc.Col([
                html.H2('Positive Material Version'),
                html.Div(id='posi_mark'),
                html.Div(dcc.Graph(id='posi_graph'))
            ],
                    width=6),
            dbc.Col([
                html.H2('Negative Material Version'),
                html.Div(id='nega_mark'),
                html.Div(dcc.Graph(id='nega_graph'))
            ],
                    width=6),
        ])
    ])
])  #end layout


#expt1 dropdown functionality
@app.callback([
    dash.dependencies.Output('unexp_graph', 'figure'),
    dash.dependencies.Output('goal_fail_graph', 'figure'),
    dash.dependencies.Output('unexp_bad_graph', 'figure'),
    dash.dependencies.Output('unexp_good_graph', 'figure'),
    dash.dependencies.Output('expt1_mark', 'children')
], dash.dependencies.Input('expt1_material_dropdown', 'value'))
def update_figure(selected_material):

    unexp_fig = val_freq_plot(
        expt1_materials_forChi,
        selected_material,
        'UNEXP',
        expt1,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)
    goal_fail_fig = val_freq_plot(
        expt1_materials_forChi,
        selected_material,
        'GOAL-FAIL',
        expt1,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)
    unexp_bad_fig = val_freq_plot(
        expt1_materials_forChi,
        selected_material,
        'UNEXP-BAD',
        expt1,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)
    unexp_good_fig = val_freq_plot(
        expt1_materials_forChi,
        selected_material,
        'UNEXP-GOOD',
        expt1,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)

    expt1_mark = dcc.Markdown(expt1_dict_scenarios[selected_material])

    return unexp_fig, goal_fail_fig, unexp_bad_fig, unexp_good_fig, expt1_mark


#expt2 dropdown functionality
@app.callback([
    dash.dependencies.Output('unexp_norm_graph', 'figure'),
    dash.dependencies.Output('unexp_bizarre_graph', 'figure'),
    dash.dependencies.Output('unexp_bad_norm_graph', 'figure'),
    dash.dependencies.Output('unexp_bad_bizarre_graph', 'figure'),
    dash.dependencies.Output('unexp_good_norm_graph', 'figure'),
    dash.dependencies.Output('unexp_good_bizarre_graph', 'figure'),
    dash.dependencies.Output('expt2_mark', 'children'),
], dash.dependencies.Input('expt2_material_dropdown', 'value'))
def update_figure(selected_material):

    unexp_norm_fig = val_freq_plot(
        expt2_materials_forChi,
        selected_material,
        'UNEXP-normal',
        expt2,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)
    unexp_bizarre_fig = val_freq_plot(
        expt2_materials_forChi,
        selected_material,
        'UNEXP-bizarre',
        expt2,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)
    unexp_bad_norm_fig = val_freq_plot(
        expt2_materials_forChi,
        selected_material,
        'UNEXP-BAD-normal',
        expt2,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)
    unexp_bad_bizarre_fig = val_freq_plot(
        expt2_materials_forChi,
        selected_material,
        'UNEXP-BAD-bizarre',
        expt2,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)

    unexp_good_norm_fig = val_freq_plot(
        expt2_materials_forChi,
        selected_material,
        'UNEXP-GOOD-normal',
        expt2,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)
    unexp_good_bizarre_fig = val_freq_plot(
        expt2_materials_forChi,
        selected_material,
        'UNEXP-GOOD-bizarre',
        expt2,
        8,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=32,
        h=45,
        brace_space_left=-0.3,
        brace_space_right=.5)

    expt2_mark = dcc.Markdown(expt1_dict_scenarios[selected_material])

    return unexp_norm_fig, unexp_bizarre_fig, unexp_bad_norm_fig, unexp_bad_bizarre_fig, unexp_good_norm_fig, unexp_good_bizarre_fig, expt2_mark


#expt3 dropdown functionality
@app.callback([
    dash.dependencies.Output('posi_graph', 'figure'),
    dash.dependencies.Output('nega_graph', 'figure'),
    dash.dependencies.Output('posi_mark', 'children'),
    dash.dependencies.Output('nega_mark', 'children'),
], dash.dependencies.Input('expt3_material_dropdown', 'value'))
def update_figure(selected_material):

    pos_fig = val_freq_plot(
        expt3_materials_forChi,
        selected_material,
        'pos_mat',
        expt3,
        10,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=30,
        h=40,
        brace_space_left=-0.3,
        brace_space_right=.5)
    neg_fig = val_freq_plot(
        expt3_materials_forChi,
        selected_material,
        'neg_mat',
        expt3,
        10,
        #                   title= str(mat+" "+cond),
        x_block_size=10,
        y_block_size=10,
        gap_size=0.5,
        w=30,
        h=40,
        brace_space_left=-0.3,
        brace_space_right=.5)

    pos_mark = dcc.Markdown(dict_scenarios[selected_material]['pos'])
    neg_mark = dcc.Markdown(dict_scenarios[selected_material]['neg'])

    return pos_fig, neg_fig, pos_mark, neg_mark


app.run_server(mode='inline', host='127.0.0.1', port=8052, debug=False)

/usr/local/lib/python3.7/dist-packages/dash/resources.py:72: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



<IPython.core.display.Javascript object>